In [77]:
import requests 
import pandas as pd
import numpy as np
import time
import logging
import re

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from bs4 import BeautifulSoup

In [78]:
logging.basicConfig(filename='lol_data.log', 
                    encoding='utf-8', 
                    level=logging.DEBUG)

In [79]:
#function that gets the html of all solo queue matches from league of Graphs of a given summoner
def lographs_htmlScrapper(summoner):
        driver = webdriver.Edge('C:\Git Projects\edgedriver_win64\msedgedriver.exe')
        driver.get("https://www.leagueofgraphs.com/summoner/lan/" + summoner)
        
        menu = driver.find_element_by_css_selector('a[data-dropdown="drop-queueTypes-recentgames"]')
        menu_item = driver.find_element_by_css_selector('a[data-additional-queue-id="420"]')
        ActionChains(driver).click(menu).pause(1).click(menu_item).pause(1)\
                            .send_keys(Keys.END).pause(np.random.randint(4)).perform()        #select option from menu
                    #while button exists get more matches 
        button_css = 'button[class="see_more_ajax_button"]'
        logging.debug("Getting data from the page..")
        while(True): 
            try:            
                button = driver.find_element_by_css_selector(button_css)
                ActionChains(driver).click(button).pause(np.random.randint(4)).send_keys(Keys.END).pause(1).perform()
            except:
                print(f'{summoner}: No more matches')
                break
                
        page_source = driver.page_source
        driver.close()
        return page_source

In [80]:
#function that takes a list of summoner names and creates a list of dataframes for each summoner in the lis
def create_dataframe(summoners):
    match_history = []
    logging.debug("Creating a dataframe for each summoner...")
    for summoner in summoners:
            html = lographs_htmlScrapper(summoner) #getting the pagesource of the player
            matches = pd.read_html(html)[0] #there are many tables in the page, we only want the first
            matches = matches.iloc[:,[2,4]] #take the columns we need
            matches.insert(0,'Summoner Name', str(summoner), True)
            matches.columns = ['Summoner Name','Result','Score']
            match_history.append(matches)
            
    return match_history

In [81]:
#As re.findall returns a list we need to get the data of each list, this functions does that
def data_from_list(lst):
    temp = []
    logging.debug("Getting the data from the lists...")
    for a in lst:
        for b in a:
            temp.append(re.sub("\s+","",b))
    return temp

In [82]:
#the function create_dataframe only create a df without separating the info, we do that here
def df_cleaning(df):
    results = [item.lstrip() for item in df['Result'] if type(item) == str]
    score = [item.lstrip() for item in df['Score'] if type(item) == str]
    status = []
    date = []
    duration= []
    kda =[]
    cs = []
    k = []
    d=[]
    a=[]
    s=[]
    
    logging.debug("Getting the data from the dataframes and creating columns for each data...")
    for result,score in zip(results,score):
        status.append(re.findall(r'^(Defeat|Victory|Remake)',result))
        date.append(re.findall(r'\d+ days ago',result))
        duration.append(re.findall(r'\d+min',result))
        
        s.append(re.findall(r'\d+\s+/\s+\d+\s+/\s+\d+',score))
        cs.append(re.findall(r'\d+ CS',score))
                   
    status = data_from_list(status)
    date = data_from_list(date)
    duration= data_from_list(duration)
    s = data_from_list(s)
    cs =  data_from_list(cs)
    
    for item in s:
        temp = re.findall(r'\d+',item)
        x=int(temp[0])
        y=int(temp[1])
        z=int(temp[2])
        k.append(x)
        d.append(y)
        a.append(z)
        if y ==0:        
            kda.append(round(((x+z)/1),2))
        else:
            kda.append(round(((x+z)/y),2))
        
    new_df = pd.DataFrame(list(zip(status,date,duration,k,d,a,kda,cs))) 
    new_df.insert(0,'Summoner Name', df['Summoner Name'], True)
    new_df.columns = ["Summoner Name","W/L", "Date" , "Duration" ,"K" ,"D","A","KDA", "CS"]
    new_df.drop(new_df[new_df['W/L'] == "Remake"].index, inplace = True)
    new_df = new_df.reset_index(drop=True)
    return new_df

In [83]:
#function that takes a list of dataframes, and a list of names and saves each data frame with the names
def save_dataframes(dataframes, names):
    for dataframe, name in zip(dataframes, names):
        if ' ' in name:
            name = re.sub(' ','_',name)           
        dataframe.to_csv(f'match_histories/{name}_soloq_match_history.csv', index = False)

In [84]:
def get_matchistories(summoners):
    match_history = create_dataframe(summoners)
    clean_match_history = []
    df = pd.DataFrame()

    for match in match_history: #cleaning dataframes
        m = df_cleaning(match)
        clean_match_history.append(m)
        df = df.append(m).reset_index(drop=True)
     
    try: #saving those dataframes
        save_dataframes(clean_match_history, summoners)
        logging.info('Dataframes saved!')
        df.to_csv(f'match_histories/all_soloq_match_history.csv', index = False)
    except:
        logging.exception('An Exception ocurred:')

# And it all comes down to this

In [85]:
summoners = ["Lopso","Cosa","WeIinton Quiw", "Rudon", "El Brayayin", 
             "Twitch Bejjaniii", "Eipa", "Don Miguel", "Aggression", "DiaCraOne", "IvanRomeroEnFB",
            "William Kurt", "Sexy Muffin", " T O X I C B O Y", "5568882336", "sneik9210", "lumaxile", 
             "skarskarSkarner", "Lixiu", "De Dya", "Rage", "EI ritardo"]

In [87]:
#recibe una lista
get_matchistories(summoners)

EI ritardo: No more matches
